# TORSO-21 Dataset

In [ ]:
import os
import numpy as np
import cv2

val2class = {0:0, 127:1, 255:2}
class2names = {0: "background", 1:"lines", 2:"fields"}

# Train set

In [ ]:
import yaml

train_annotations = "data/reality/train/annotations.yaml"
train_data = None

with open(train_annotations, "r") as f:
    train_data = yaml.safe_load(f)
        


In [ ]:
import fiftyone as fo
from tqdm.notebook import tqdm

train_samples = []
train_image_folder = "data/reality/train/images"
train_segmentation_folder = "data/reality/train/segmentations_fix"

for image_name in tqdm(train_data["images"]):
    
    image_path = os.path.join(train_image_folder, image_name)
    seg_img_path = os.path.join(train_segmentation_folder, image_name[:-4]+".png")
    
    if not os.path.exists(image_path):
        print(image_path)
        continue
    
    data = train_data["images"][image_name]
    metadata = data['metadata']
    image_metadata = fo.ImageMetadata.build_for(image_path)
    metadata = {**metadata, **image_metadata.to_dict()}
    metadata = fo.Metadata.from_dict(metadata)

    # Create Fiftyone sample
    sample = fo.Sample(
        filepath=image_path,
        metadata=metadata,
        id=data['id'],
        tags=['train']
    )

    # dict_keys(['annotations', 'height', 'id', 'metadata', 'width'])
    detections = []
    polylines = []
    keypoints = []

    for ann in data['annotations']:

        if ann['in_image']:
            
            # box annotations for ball, obstacle, robot
            if ann['type'] in ['obstacle', 'robot', 'ball']:
                for i in range(0, len(ann['vector']), 2):
                    bbox = ann['vector'][i] + ann['vector'][i+1]
                    x1, y1, w, h = bbox[0], bbox[1], bbox[2]-bbox[0], bbox[3]-bbox[1]
                    detections.append(
                        fo.Detection(
                        label=ann['type'],
                        bounding_box=[x1/image_metadata['width'],y1/image_metadata['height'],w/image_metadata['width'],h/image_metadata['height']],
                        blurred=ann['blurred'],
                        concealed=ann['concealed'],
                        )
                    )
                    
            # Polygons for goalpost
            elif ann['type'] in ['goalpost']:
                points = []
                for i in range(0, len(ann['vector'])):
                    p = ann['vector'][i]
                    p[0] /= image_metadata['width']
                    p[1] /= image_metadata['height']
                    points.append(list(p))
                    
                # A closed, filled polygon with a label
                polylines.append(
                    fo.Polyline(
                        label=ann['type'],
                        points=[points],
                        closed=True,
                        filled=True,
                        blurred=ann['blurred'],
                        concealed=ann['concealed'],

                    )
                )
                
            # Two points center (x,y)
            elif ann['type'] in ['X-Intersection', 'L-Intersection', 'T-Intersection']:
                points = []
                for i in range(0, len(ann['vector'])):
                    p = ann['vector'][i]
                    p[0] /= image_metadata['width']
                    p[1] /= image_metadata['height']
                    points.append(list(p))
                    
                keypoints.append(
                    fo.Keypoint(
                        label=ann['type'],
                        points=points,
                        blurred=ann['blurred'],
                        concealed=ann['concealed'],
                    )
                )
    
    # Update to sample
    sample["detections"] = fo.Detections(detections=detections) if len(detections) > 0 else None
    sample["polylines"] = fo.Polylines(polylines=polylines) if len(polylines) > 0 else None
    sample["keypoints"] = fo.Keypoints(keypoints=keypoints) if len(keypoints) > 0 else None

    # Create segmentation mask
    if os.path.exists(seg_img_path):

        seg_image = cv2.imread(seg_img_path)
        seg_image = cv2.cvtColor(seg_image, cv2.COLOR_BGR2GRAY)

        for k,v in val2class.items():
            seg_image[seg_image == k] = v
            
        sample["segmentation"] = fo.Segmentation(mask=seg_image)
    else:
        if os.path.exists(seg_img_path.replace('segmentations_fix', 'segmentations')):
            print('found in original dir: ', seg_img_path)
        else:
            print('missing: ', seg_img_path)
    train_samples.append(sample)

  0%|          | 0/8894 [00:00<?, ?it/s]

# Test Set

In [ ]:
test_annotations = "data/reality/test/annotations.yaml"
test_data = None

with open(test_annotations, "r") as f:
    test_data = yaml.safe_load(f)

test_samples = []
test_image_folder = "data/reality/test/images"
test_segmentation_folder = "data/reality/test/segmentations_fix"

for image_name in tqdm(test_data["images"]):
    
    image_path = os.path.join(test_image_folder, image_name)
    seg_img_path = os.path.join(test_segmentation_folder, image_name[:-4]+".png")
    
    if not os.path.exists(image_path):
        print(image_path)
        continue
    
    data = test_data["images"][image_name]
    metadata = data['metadata']
    image_metadata = fo.ImageMetadata.build_for(image_path)
    metadata = {**metadata, **image_metadata.to_dict()}
    metadata = fo.Metadata.from_dict(metadata)
    
    # Create Fiftyone sample
    sample = fo.Sample(
        filepath=image_path,
        metadata=metadata,
        id=data['id'],
        tags=['test']
    )

    # dict_keys(['annotations', 'height', 'id', 'metadata', 'width'])
    detections = []
    polylines = []
    keypoints = []

    for ann in data['annotations']:

        if ann['in_image']:
            
            # box annotations for ball, obstacle, robot
            if ann['type'] in ['obstacle', 'robot', 'ball']:
                for i in range(0, len(ann['vector']), 2):
                    bbox = ann['vector'][i] + ann['vector'][i+1]
                    x1, y1, w, h = bbox[0], bbox[1], bbox[2]-bbox[0], bbox[3]-bbox[1]
                    detections.append(
                        fo.Detection(
                        label=ann['type'],
                        bounding_box=[x1/image_metadata['width'],y1/image_metadata['height'],w/image_metadata['width'],h/image_metadata['height']],
                        blurred=ann['blurred'],
                        concealed=ann['concealed'],
                        )
                    )
                    
            # Polygons for goalpost
            # TODO: Fix wrong value in FO
            elif ann['type'] in ['goalpost']:
                points = []
                for i in range(0, len(ann['vector'])):
                    p = ann['vector'][i]
                    p[0] /= image_metadata['width']
                    p[1] /= image_metadata['height']
                    points.append(list(p))
                    
                # A closed, filled polygon with a label
                polylines.append(
                    fo.Polyline(
                        label=ann['type'],
                        points=[points],
                        closed=True,
                        filled=True,
                        blurred=ann['blurred'],
                        concealed=ann['concealed'],

                    )
                )
                
            # Two points center (x,y)
            # TODO: Fix wrong value in FO
            elif ann['type'] in ['X-Intersection', 'L-Intersection', 'T-Intersection']:
                points = []
                for i in range(0, len(ann['vector'])):
                    p = ann['vector'][i]
                    p[0] /= image_metadata['width']
                    p[1] /= image_metadata['height']
                    points.append(list(p))
                    
                keypoints.append(
                    fo.Keypoint(
                        label=ann['type'],
                        points=points,
                        blurred=ann['blurred'],
                        concealed=ann['concealed'],
                    )
                )
    
    # Update to sample
    sample["detections"] = fo.Detections(detections=detections) if len(detections) > 0 else None
    sample["polylines"] = fo.Polylines(polylines=polylines) if len(polylines) > 0 else None
    sample["keypoints"] = fo.Keypoints(keypoints=keypoints) if len(keypoints) > 0 else None

    # Create segmentation mask
    if os.path.exists(seg_img_path):

        seg_image = cv2.imread(seg_img_path)
        seg_image = cv2.cvtColor(seg_image, cv2.COLOR_BGR2GRAY)

        for k,v in val2class.items():
            seg_image[seg_image == k] = v
            
        sample["segmentation"] = fo.Segmentation(mask=seg_image)
    else:
        if os.path.exists(seg_img_path.replace('segmentations_fix', 'segmentations')):
            print('found in original dir: ', seg_img_path)
        else:
            print('missing: ', seg_img_path)
    test_samples.append(sample)

  0%|          | 0/1570 [00:00<?, ?it/s]

# Create Fiftyone Dataset

In [ ]:
import fiftyone as fo

# Create an empty dataset
dataset = fo.Dataset("torso_21_dataset", overwrite=True)

# Store a class list in the dataset's info
dataset.classes = {
    "object_detections": ['obstacle', 'robot', 'ball'],
    "polylines": ["goalpost"],
    "keypoints": ['X-Intersection', 'L-Intersection', 'T-Intersection'],
}

dataset.mask_targets = {
    "ground_truth": {1:"lines", 2:"fields"},
}
    
print(dataset)
dataset.add_samples(train_samples)
dataset.add_samples(test_samples)

# Export the dataset
dataset.export(
    export_dir="./torso_21_dataset",
    dataset_type=fo.types.FiftyOneImageDetectionDataset,
    export_media="symlink"
)


# Preview with FiftyOne

In [ ]:
session = fo.launch_app(dataset)